In [2]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
import asyncio
import os

import requests
import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

from typing import Literal, Union
from pydantic.config import ConfigDict

from src.models import get_analysis, apply_async_analysis
from src.utilities import *

import openai
import instructor



In [3]:
feedbacks_df = pd.read_csv('www.darty.com.csv')
feedbacks_df

,Unnamed: 0,Title,Date,Content,Page,Username,Total reviews,Location,Rating
0,0,"Fuyez, une arnaque","February 05, 2024",2000€ d’appareils et ils ne fonctionnent pas p...,1,Violaine Becha,1 review,FR,1.0
1,1,Centrale vapeur calor défaillante au bout de 5...,"February 03, 2024",Le lundi 29 janvier 2024 je récupère une centr...,1,Isabelle bas-rhin,9 reviews,FR,1.0
2,2,Évitez !! Les actions du vendeur sont absolume...,"March 30, 2023",Évitez d'acheter auprès de ce vendeur en ligne...,1,Mikhail Shangin,3 reviews,FR,1.0
3,3,Le SAV est une catastrophe!,"January 15, 2024",Le SAV est une catastrophe!Le client ne l'est ...,1,SCD,1 review,FR,1.0
4,4,Machine a laver Bosch,"January 05, 2024",Commandée sur le site de Darty machine a laver...,1,Christiane,56 reviews,FR,5.0
...,...,...,...,...,...,...,...,...,...
9027,8963,Globalement bien,"December 01, 2011",Bonne qualité des communications téléphoniques...,449,michpastaga,12 reviews,FR,5.0
9028,8964,nouvel adherent,"September 15, 2011",je viens de souscrire aa l'offre darty interne...,449,rabekoto de princy,2 reviews,FR,5.0
9029,8965,Mamac,"September 04, 2011",service SAV LUXEMBOURG archi nul totalement i...,449,Mamac,7 reviews,BE,1.0
9030,8966,Très bon,"June 05, 2011","Je suis toujours très satisfaite de ce site, r...",449,Bailey,2 reviews,FR,5.0
